In [112]:
import requests
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
import pandas as pd
import numpy as np
from pymarc import MARCReader
import xml.etree.ElementTree as ET
import mysql.connector
from datetime import datetime

In [2]:
def modify_isbn(isbn):
    '''
    This function converts ISBN values to a string, then takes ISBN values missing '978' or '979' at the beginning and 
    inserts '978' as a placeholder for the time being. The function then returns the first 13 characters of the value (the standard length of ISBNs).DS_Store

    Parameters:
    isbn - A given ISBN value.

    Returns:
    isbn - The first 13 characters of the ISBN value.
    '''
    isbn = str(isbn)
    # Check if the first 3 characters are neither '978' nor '979', and the length is 10
    if not (isbn[:3] == '978' or isbn[:3] == '979'):
        if len(isbn) == 10:
            return '978' + isbn  # Prepend '978' if conditions are met
    return isbn[:13]  # Return the first 13 characters of ISBN as-is if conditions are not met

def add_attributes(df):
    '''
    This function adds desired features/attributes that are missing to the dataframe.

    Parameters:
    df - Dataframe that is set to be adjusted.

    Returns: None (Dataframe is updated)
    '''
    category_list = ['Title/Subtitle', 'Author', 'Copyright Date', 'Summary', 
                    'Series Name/Position', 'Genre', 'Form', 'Format', 'ISBN', 
                    'Page Count', 'Type', 'Publisher', 'Publication Year',
                    'Material Type', 'Subject', 'Lexile']
    for category in category_list:
        if category not in df.columns:
            df.loc[:,category] = np.nan

def extract_first_valid_identifier(identifier_str):
    '''
    This function parses for a single ISBN

    Parameters:
    identifier_str - String of ISBN-like values.

    Returns:
    identifier - ISBN value
    None - None if not possible
    '''
    identifiers = identifier_str.split(',')
    for identifier in identifiers:
        identifier = identifier.replace(" ", "")
        if identifier.startswith(('978', '979')):
            return identifier
    return None  # If no valid identifier found

In [129]:
def data_cleaning(df):
    df = df.astype(str)
    extra_feature_list = ['Manual Tags', 'Automatic Tags','Translator', 
        'Call Number', 'Extra', 'Notes', 'File Attachments', 'Link Attachments', 
        'Place', 'Language', 'Rights', 'Type', 'Archive', 'Archive Location', 
        'Library Catalog','Contributor', 'Attorney Agent', 'Book Author',
        'Cast Member', 'Commenter', 'Composer', 'Cosponsor', 'Counsel',
        'Interviewer', 'Producer', 'Recipient', 'Reviewed Author',
        'Scriptwriter', 'Words By', 'Guest', 'Number', 'Edition',
        'Running Time', 'Scale', 'Medium', 'Artwork Size', 'Filing Date',
        'Application Number', 'Assignee', 'Issuing Authority', 'Country',
        'Meeting Name', 'Conference Name', 'Court', 'References', 'Reporter',
        'Legal Status', 'Priority Numbers', 'Programming Language', 'Version',
        'System', 'Code', 'Code Number', 'Section', 'Session', 'Committee',
        'History', 'Legislative Body', 'ISSN']
    columns_to_drop = []
    for extra_feature in df.columns:
        if extra_feature in extra_feature_list:
            columns_to_drop.append(extra_feature)
    df.drop(columns=columns_to_drop, inplace=True)
    add_attributes(df)
    display(df)
    nan_df = df[df['ISBN'].isna()]
    isbn_df = df[~df['ISBN'].isna()]

    isbn_df.loc[:, 'ISBN'] = isbn_df['ISBN'].str.replace('-', '', regex=False)
    isbn_df.loc[:, 'ISBN'] = isbn_df['ISBN'].apply(modify_isbn)

    broken_isbn_df = isbn_df[~isbn_df['ISBN'].str[:3].isin(['978', '979'])]
    whole_isbn_df = isbn_df[isbn_df['ISBN'].str[:3].isin(['978', '979'])]

    broken_isbn_df.loc[:,'ISBN'] = broken_isbn_df['ISBN'].apply(lambda x: '978' + str(x)[:10]) # Is 978 the right choice here??

    isbn_df = pd.concat([broken_isbn_df, whole_isbn_df])

    isbn_df = isbn_df.astype(str)

    # # Drop duplicates based on relevant columns while keeping the first occurrence
    isbn_df_unique = isbn_df.drop_duplicates(subset=['ISBN'], keep='first')

    # # Group by 'ISBN' and combine the 'Subject' values
    isbn_df_unique = isbn_df_unique.groupby('ISBN').agg({
        'Series Title': 'first',  # Keep the first value (they are the same for each ISBN)
        'Title/Subtitle': 'first',  # Same as above
        'ISSN': 'first',  # Same as above
        'Author': 'first',  # Same as above
        'Publication Year': 'first',  # Same as above
        'Publisher': 'first',  # Same as above
        'Material Type': 'first',  # Same as above
        'Lexile': 'first',  # Same as above
        'Subject': ', '.join  # Combine all unique 'Subject' values
    }).reset_index()

    for isbn in isbn_df_unique['ISBN']:
        if pd.isna(isbn):
            continue  # Skip if ISBN is NaN
        url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
        response = requests.get(url)
        if response.status_code == 200: # Confirm request worked/is available
            book_data = response.json()
            if 'items' in book_data:
            # Extract book details from the API response
                book_info = book_data['items'][0]['volumeInfo']
                # Missing:
                # Series name and position
                # Form
                # Format
                book_details = {
                    'ISBN#': isbn,
                    'Title': book_info.get('title', np.nan),
                    'Subtitle': np.nan, # Not available
                    'Authors': ', '.join(book_info.get('authors', ['N/A'])),
                    'Publisher': book_info.get('publisher', np.nan),
                    'PublishedDate': book_info.get('publishedDate', np.nan),
                    'CopyrightDate': np.nan, # Not available
                    'Summary': book_info.get('description', 'No description available'), # NEED
                    'Genre': ', '.join(book_info.get('categories', ['N/A'])), # NEED
                    'PageCount': book_info.get('pageCount', np.nan), # NEED
                    'Type': book_info.get('printType', np.nan), # Need
                    'Categories': book_info.get('Categories', np.nan)
                    # Type of book - ficiton, nonfiction, blended
                }
                # Accessing the correct row using ISBN in the DataFrame
                row_index = isbn_df_unique[isbn_df_unique['ISBN'] == isbn].index
                
                if not row_index.empty:  # Check if a matching ISBN was found
                    row_index = row_index[0]  # Get the first (and expected only) match

                    # Only update if the current value is NaN
                    if pd.isna(isbn_df_unique.loc[row_index, 'Title/Subtitle']):
                        isbn_df_unique.loc[row_index, 'Title/Subtitle'] = book_details['Title']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Author']):
                        isbn_df_unique.loc[row_index, 'Author'] = book_details['Authors']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Publication Year']):
                        isbn_df_unique.loc[row_index, 'Publication Year'] = book_details['PublishedDate']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Publisher']):
                        isbn_df_unique.loc[row_index, 'Publisher'] = book_details['Publisher']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Material Type']):
                        isbn_df_unique.loc[row_index, 'Material Type'] = book_details['Type']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Subject']):
                        isbn_df_unique.loc[row_index, 'Subject'] = book_details['Genre']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Summary']):
                        isbn_df_unique.loc[row_index, 'Summary'] = book_details['Summary']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Page Count']):
                        isbn_df_unique.loc[row_index, 'Page Count'] = book_details['PageCount']

    complete_df = pd.concat([isbn_df_unique, nan_df])
    return complete_df

In [3]:
# /Users/rwebb/Desktop/ReMo/CRWReportJob148737.xlsx
# /Users/rwebb/Desktop/ReMo/ONIX/LEEANDLOW_20210707.xml
# /Users/rwebb/Desktop/ReMo/ONIX/Lerner_Print_ONIX_20240104104306.xml

In [4]:
# Handle excel (.xlsx) files, scraping ISBNs --> Learn to convert other files
# Use API to fill data points
# Fill gaps with excel

## LEEANDLOW xml

In [97]:
# /Users/rwebb/Desktop/ReMo/ONIX/LEEANDLOW_20210707.xml
# handling LEEANDLOW xml file

# filename = input('Enter File Name: ')
filename = '/Users/rwebb/Desktop/ReMo/ONIX/LEEANDLOW_20210707.xml'

# Define the XML namespace
namespace = {'onix': 'http://ns.editeur.org/onix/3.0/reference'}

# Parse the XML file
tree = ET.parse(filename)
root = tree.getroot()

# Create a list to store dictionaries representing each book
books_list = []

# Iterate through each <Product> element, considering the namespace
for book_elem in root.findall('.//onix:Product', namespace):
    book_dict = {}
    
    # Extracting the RecordReference from the Product element
    book_dict['RecordReference'] = book_elem.find('onix:RecordReference', namespace).text if book_elem.find('onix:RecordReference', namespace) is not None else None
    
    # Extracting IDValue from ProductIdentifier elements
    product_ids = []
    for product_id_elem in book_elem.findall('onix:ProductIdentifier', namespace):
        id_value = product_id_elem.find('onix:IDValue', namespace).text if product_id_elem.find('onix:IDValue', namespace) is not None else None
        product_ids.append(id_value)
    book_dict['ProductIdentifiers'] = ', '.join(product_ids)  # Join multiple IDs with commas if there are multiple ProductIdentifiers
    
    # Extracting TitleText from TitleDetail
    title_text = book_elem.find('.//onix:TitleDetail/onix:TitleElement/onix:TitleText', namespace)
    book_dict['TitleText'] = title_text.text if title_text is not None else None
    
    # Extracting PersonName from Contributor
    contributor_names = []
    for contributor_elem in book_elem.findall('.//onix:Contributor', namespace):
        person_name = contributor_elem.find('onix:PersonName', namespace)
        if person_name is not None:
            contributor_names.append(person_name.text)
    book_dict['ContributorNames'] = ', '.join(contributor_names)  # Join multiple names if there are multiple contributors
    
    # Extracting SubjectHeadingText from Subject
    subject_heading = book_elem.find('.//onix:Subject/onix:SubjectHeadingText', namespace)
    book_dict['SubjectHeadingText'] = subject_heading.text if subject_heading is not None else None
    
    # Extracting Date from PublishingDate (Publish Date)
    publishing_date = book_elem.find('.//onix:PublishingDetail/onix:PublishingDate/onix:Date', namespace)
    book_dict['PublishingDate'] = publishing_date.text if publishing_date is not None else None

    # Append the dictionary to the list
    books_list.append(book_dict)

# Create a Pandas DataFrame from the list of dictionaries
leelow_df = pd.DataFrame(books_list)

# Print the resulting DataFrame and books_list for debugging
leelow_df

,RecordReference,ProductIdentifiers,TitleText,ContributorNames,SubjectHeadingText,PublishingDate
0,Miosotis Flores Never Forgets,"1643790668, 9781643790664, 9781643790664, b032...",Miosotis Flores Never Forgets,Hilda Eunice Burgos,books for middle school girls;ages 9-12;rescue...,20211026
1,Miosotis Flores Never Forgets,"9781643790657, 9781643790657, 153dc26a-84ae-4a...",Miosotis Flores Never Forgets,Hilda Eunice Burgos,books for middle school girls;ages 9-12;rescue...,20211026
2,Black Was the Ink,"1643794329, 9781643794327, 9781643794327, b5d7...",Black Was the Ink,"Michelle Coles, Justin Johnson",Reconstruction;Black history;Black teens;ownvo...,20211012
3,Black Was the Ink,"9781643794310, 9781643794310, f3ead083-825e-43...",Black Was the Ink,"Michelle Coles, Justin Johnson",Reconstruction;Black history;Black teens;ownvo...,20211012
4,"Xochitl and the Flowers / Xóchitl, la Niña de ...","0892393602, 9780892393602, 9780892393602, 61f1...","Xochitl and the Flowers / Xóchitl, la Niña de ...","Jorge Argueta, Carl Angel",Contemporary Stories;Community;Family/Intergen...,20140801
5,The Shadow Prince,"9781643794280, 9781643794280, 239b5690-9e55-4d...",None,David Anthony Durham,Ancient Egypt;Egyptian gods;solarpunk;Black he...,20211019
6,Boys of the Beast,"164379096X, 9781643790961, 9781643790961, da49...",Boys of the Beast,Monica Zepeda,road trip;school shooting;Latinx teen books;tr...,20220101
7,Boys of the Beast,"9781643790954, 9781643790954, 9df85f1e-efd4-41...",Boys of the Beast,Monica Zepeda,road trip;school shooting;Latinx teen books;tr...,20220208


In [98]:
# Data cleaning
leelow_df['ISBN'] = leelow_df['ProductIdentifiers'].apply(extract_first_valid_identifier)
leelow_df.rename(columns={'TitleText':'Title/Subtitle', 'ContributorNames':'Author'}, inplace=True)
leelow_df.drop(columns=['ProductIdentifiers'], inplace=True)
add_attributes(leelow_df)
leelow_df

,RecordReference,Title/Subtitle,Author,SubjectHeadingText,PublishingDate,ISBN,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type,Publisher,Publication Year,Material Type,Subject,Lexile
0,Miosotis Flores Never Forgets,Miosotis Flores Never Forgets,Hilda Eunice Burgos,books for middle school girls;ages 9-12;rescue...,20211026,9781643790664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Miosotis Flores Never Forgets,Miosotis Flores Never Forgets,Hilda Eunice Burgos,books for middle school girls;ages 9-12;rescue...,20211026,9781643790657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Black Was the Ink,Black Was the Ink,"Michelle Coles, Justin Johnson",Reconstruction;Black history;Black teens;ownvo...,20211012,9781643794327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Black Was the Ink,Black Was the Ink,"Michelle Coles, Justin Johnson",Reconstruction;Black history;Black teens;ownvo...,20211012,9781643794310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Xochitl and the Flowers / Xóchitl, la Niña de ...","Xochitl and the Flowers / Xóchitl, la Niña de ...","Jorge Argueta, Carl Angel",Contemporary Stories;Community;Family/Intergen...,20140801,9780892393602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,The Shadow Prince,None,David Anthony Durham,Ancient Egypt;Egyptian gods;solarpunk;Black he...,20211019,9781643794280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Boys of the Beast,Boys of the Beast,Monica Zepeda,road trip;school shooting;Latinx teen books;tr...,20220101,9781643790961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Boys of the Beast,Boys of the Beast,Monica Zepeda,road trip;school shooting;Latinx teen books;tr...,20220208,9781643790954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
for isbn in leelow_df['ISBN']:
    if pd.isna(isbn):
        continue  # Skip if ISBN is NaN
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    if response.status_code == 200: # Confirm request worked/is available
        book_data = response.json()
        if 'items' in book_data:
        # Extract book details from the API response
            book_info = book_data['items'][0]['volumeInfo']
            # Missing:
            # Series name and position
            # Form
            # Format
            book_details = {
                'ISBN#': isbn,
                'Title': book_info.get('title', np.nan),
                'Subtitle': np.nan, # Not available
                'Authors': ', '.join(book_info.get('authors', ['N/A'])),
                'Publisher': book_info.get('publisher', np.nan),
                'PublishedDate': book_info.get('publishedDate', np.nan),
                'CopyrightDate': np.nan, # Not available
                'Summary': book_info.get('description', 'No description available'), # NEED
                'Genre': ', '.join(book_info.get('categories', ['N/A'])), # NEED
                'PageCount': book_info.get('pageCount', np.nan), # NEED
                'Type': book_info.get('printType', np.nan), # Need
                'Categories': book_info.get('Categories', np.nan)
                # Type of book - ficiton, nonfiction, blended
            }
            # Accessing the correct row using ISBN in the DataFrame
            row_index = leelow_df[leelow_df['ISBN'] == isbn].index
            
            if not row_index.empty:  # Check if a matching ISBN was found
                row_index = row_index[0]  # Get the first (and expected only) match

                # Only update if the current value is NaN
                if pd.isna(leelow_df.loc[row_index, 'Title/Subtitle']):
                    leelow_df.loc[row_index, 'Title/Subtitle'] = book_details['Title']
                if pd.isna(leelow_df.loc[row_index, 'Author']):
                    leelow_df.loc[row_index, 'Author'] = book_details['Authors']
                if pd.isna(leelow_df.loc[row_index, 'Publication Year']):
                    leelow_df.loc[row_index, 'Publication Year'] = book_details['PublishedDate']
                if pd.isna(leelow_df.loc[row_index, 'Publisher']):
                    leelow_df.loc[row_index, 'Publisher'] = book_details['Publisher']
                if pd.isna(leelow_df.loc[row_index, 'Material Type']):
                    leelow_df.loc[row_index, 'Material Type'] = book_details['Type']
                if pd.isna(leelow_df.loc[row_index, 'Subject']):
                    leelow_df.loc[row_index, 'Subject'] = book_details['Genre']
                if pd.isna(leelow_df.loc[row_index, 'Summary']):
                    leelow_df.loc[row_index, 'Summary'] = book_details['Summary']
                if pd.isna(leelow_df.loc[row_index, 'Page Count']):
                    leelow_df.loc[row_index, 'Page Count'] = book_details['PageCount']

/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_1985/3731187849.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2021' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  leelow_df.loc[row_index, 'Publication Year'] = book_details['PublishedDate']
/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_1985/3731187849.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tu Books' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  leelow_df.loc[row_index, 'Publisher'] = book_details['Publisher']
/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_1985/3731187849.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'BOOK' has dtype incompatib

In [100]:
leelow_df

,RecordReference,Title/Subtitle,Author,SubjectHeadingText,PublishingDate,ISBN,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type,Publisher,Publication Year,Material Type,Subject,Lexile
0,Miosotis Flores Never Forgets,Miosotis Flores Never Forgets,Hilda Eunice Burgos,books for middle school girls;ages 9-12;rescue...,20211026,9781643790664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Miosotis Flores Never Forgets,Miosotis Flores Never Forgets,Hilda Eunice Burgos,books for middle school girls;ages 9-12;rescue...,20211026,9781643790657,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Black Was the Ink,Black Was the Ink,"Michelle Coles, Justin Johnson",Reconstruction;Black history;Black teens;ownvo...,20211012,9781643794327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Black Was the Ink,Black Was the Ink,"Michelle Coles, Justin Johnson",Reconstruction;Black history;Black teens;ownvo...,20211012,9781643794310,NaN,"Through the help of a ghostly ancestor, sixtee...",NaN,NaN,NaN,NaN,0.0,NaN,Tu Books,2021,BOOK,JUVENILE FICTION,NaN
4,"Xochitl and the Flowers / Xóchitl, la Niña de ...","Xochitl and the Flowers / Xóchitl, la Niña de ...","Jorge Argueta, Carl Angel",Contemporary Stories;Community;Family/Intergen...,20140801,9780892393602,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,The Shadow Prince,The Shadow Prince,David Anthony Durham,Ancient Egypt;Egyptian gods;solarpunk;Black he...,20211019,9781643794280,NaN,In this middle grade solarpunk novel set in an...,NaN,NaN,NaN,NaN,0.0,NaN,Tu Books,2021,BOOK,JUVENILE FICTION,NaN
6,Boys of the Beast,Boys of the Beast,Monica Zepeda,road trip;school shooting;Latinx teen books;tr...,20220101,9781643790961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Boys of the Beast,Boys of the Beast,Monica Zepeda,road trip;school shooting;Latinx teen books;tr...,20220208,9781643790954,NaN,"""Three teenage boy cousins on a road trip thro...",NaN,NaN,NaN,NaN,304.0,NaN,NaN,2022,BOOK,Young Adult Fiction,NaN


In [101]:
leelow_df['PublishingDate'] = pd.to_datetime(leelow_df['PublishingDate'])

In [102]:
leelow_sql_input = leelow_df.rename(columns={'RecordReference':'title', "Author":"author", 
                                             "SubjectHeadingText":"subgenre", "Lexile":"lexileLevel", 
                                             "PublishingDate":"copyrightDate", "Summary":"summary", 
                                             "Material Type":"type", "Form":"form", "Subject":'genre', 
                                             'ISBN':'isbn', "Publisher":'publisher', 
                                             'Series Name/Position':'seriesName/Position', 'Page Count':'pageCount'})
leelow_sql_input = leelow_sql_input[['isbn','title',"author",'genre',"subgenre","lexileLevel",
                                    'publisher', "copyrightDate","summary","type","form", 
                                    'seriesName/Position', 'pageCount']]

In [103]:
leelow_sql_input

,isbn,title,author,genre,subgenre,lexileLevel,publisher,copyrightDate,summary,type,form,seriesName/Position,pageCount
0,9781643790664,Miosotis Flores Never Forgets,Hilda Eunice Burgos,NaN,books for middle school girls;ages 9-12;rescue...,NaN,NaN,2021-10-26,NaN,NaN,NaN,NaN,NaN
1,9781643790657,Miosotis Flores Never Forgets,Hilda Eunice Burgos,NaN,books for middle school girls;ages 9-12;rescue...,NaN,NaN,2021-10-26,NaN,NaN,NaN,NaN,NaN
2,9781643794327,Black Was the Ink,"Michelle Coles, Justin Johnson",NaN,Reconstruction;Black history;Black teens;ownvo...,NaN,NaN,2021-10-12,NaN,NaN,NaN,NaN,NaN
3,9781643794310,Black Was the Ink,"Michelle Coles, Justin Johnson",JUVENILE FICTION,Reconstruction;Black history;Black teens;ownvo...,NaN,Tu Books,2021-10-12,"Through the help of a ghostly ancestor, sixtee...",BOOK,NaN,NaN,0.0
4,9780892393602,"Xochitl and the Flowers / Xóchitl, la Niña de ...","Jorge Argueta, Carl Angel",NaN,Contemporary Stories;Community;Family/Intergen...,NaN,NaN,2014-08-01,NaN,NaN,NaN,NaN,NaN
5,9781643794280,The Shadow Prince,David Anthony Durham,JUVENILE FICTION,Ancient Egypt;Egyptian gods;solarpunk;Black he...,NaN,Tu Books,2021-10-19,In this middle grade solarpunk novel set in an...,BOOK,NaN,NaN,0.0
6,9781643790961,Boys of the Beast,Monica Zepeda,NaN,road trip;school shooting;Latinx teen books;tr...,NaN,NaN,2022-01-01,NaN,NaN,NaN,NaN,NaN
7,9781643790954,Boys of the Beast,Monica Zepeda,Young Adult Fiction,road trip;school shooting;Latinx teen books;tr...,NaN,NaN,2022-02-08,"""Three teenage boy cousins on a road trip thro...",BOOK,NaN,NaN,304.0


In [106]:
leelow_sql_input = leelow_sql_input.replace(np.nan, 'None')
leelow_sql_input

,isbn,title,author,genre,subgenre,lexileLevel,publisher,copyrightDate,summary,type,form,seriesName/Position,pageCount
0,9781643790664,Miosotis Flores Never Forgets,Hilda Eunice Burgos,None,books for middle school girls;ages 9-12;rescue...,None,None,2021-10-26,None,None,None,None,None
1,9781643790657,Miosotis Flores Never Forgets,Hilda Eunice Burgos,None,books for middle school girls;ages 9-12;rescue...,None,None,2021-10-26,None,None,None,None,None
2,9781643794327,Black Was the Ink,"Michelle Coles, Justin Johnson",None,Reconstruction;Black history;Black teens;ownvo...,None,None,2021-10-12,None,None,None,None,None
3,9781643794310,Black Was the Ink,"Michelle Coles, Justin Johnson",JUVENILE FICTION,Reconstruction;Black history;Black teens;ownvo...,None,Tu Books,2021-10-12,"Through the help of a ghostly ancestor, sixtee...",BOOK,None,None,0.0
4,9780892393602,"Xochitl and the Flowers / Xóchitl, la Niña de ...","Jorge Argueta, Carl Angel",None,Contemporary Stories;Community;Family/Intergen...,None,None,2014-08-01,None,None,None,None,None
5,9781643794280,The Shadow Prince,David Anthony Durham,JUVENILE FICTION,Ancient Egypt;Egyptian gods;solarpunk;Black he...,None,Tu Books,2021-10-19,In this middle grade solarpunk novel set in an...,BOOK,None,None,0.0
6,9781643790961,Boys of the Beast,Monica Zepeda,None,road trip;school shooting;Latinx teen books;tr...,None,None,2022-01-01,None,None,None,None,None
7,9781643790954,Boys of the Beast,Monica Zepeda,Young Adult Fiction,road trip;school shooting;Latinx teen books;tr...,None,None,2022-02-08,"""Three teenage boy cousins on a road trip thro...",BOOK,None,None,304.0


## Connect to DB and create tables/insert data

In [107]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    database = "Remo"
)

if mydb.is_connected():
    print("Successfully connected to the database")
else:
    print("Failed to connect to the database")

Successfully connected to the database


In [108]:
# Create a cursor object using the connection
cursor = mydb.cursor()

In [109]:
cursor.execute("CREATE DATABASE IF NOT EXISTS Remo")

In [110]:
# Create a table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS LastTest (
        isbn VARCHAR(20) NULL,
        title VARCHAR(80) NULL, 
        author VARCHAR(80) NULL, 
        genre VARCHAR(80) NULL, 
        subgenre VARCHAR(500) NULL, 
        awards VARCHAR(150) NULL, 
        contributors VARCHAR(300) NULL, 
        lexileLevel VARCHAR(30) NULL, 
        copyrightDate VARCHAR(15) NULL, 
        summary TEXT NULL, 
        type VARCHAR(50) NULL, 
        seriesName VARCHAR(80) NULL, 
        seriesNumber VARCHAR(4) NULL, 
        form VARCHAR(50) NULL, 
        subtitle VARCHAR(80) NULL
    )
""")

# Commit the transaction
mydb.commit()

In [111]:
leelow_sql_input['subgenre'] = leelow_sql_input['subgenre'].apply(lambda x: x[:500] if isinstance(x, str) else x)
leelow_sql_input

,isbn,title,author,genre,subgenre,lexileLevel,publisher,copyrightDate,summary,type,form,seriesName/Position,pageCount
0,9781643790664,Miosotis Flores Never Forgets,Hilda Eunice Burgos,None,books for middle school girls;ages 9-12;rescue...,None,None,2021-10-26,None,None,None,None,None
1,9781643790657,Miosotis Flores Never Forgets,Hilda Eunice Burgos,None,books for middle school girls;ages 9-12;rescue...,None,None,2021-10-26,None,None,None,None,None
2,9781643794327,Black Was the Ink,"Michelle Coles, Justin Johnson",None,Reconstruction;Black history;Black teens;ownvo...,None,None,2021-10-12,None,None,None,None,None
3,9781643794310,Black Was the Ink,"Michelle Coles, Justin Johnson",JUVENILE FICTION,Reconstruction;Black history;Black teens;ownvo...,None,Tu Books,2021-10-12,"Through the help of a ghostly ancestor, sixtee...",BOOK,None,None,0.0
4,9780892393602,"Xochitl and the Flowers / Xóchitl, la Niña de ...","Jorge Argueta, Carl Angel",None,Contemporary Stories;Community;Family/Intergen...,None,None,2014-08-01,None,None,None,None,None
5,9781643794280,The Shadow Prince,David Anthony Durham,JUVENILE FICTION,Ancient Egypt;Egyptian gods;solarpunk;Black he...,None,Tu Books,2021-10-19,In this middle grade solarpunk novel set in an...,BOOK,None,None,0.0
6,9781643790961,Boys of the Beast,Monica Zepeda,None,road trip;school shooting;Latinx teen books;tr...,None,None,2022-01-01,None,None,None,None,None
7,9781643790954,Boys of the Beast,Monica Zepeda,Young Adult Fiction,road trip;school shooting;Latinx teen books;tr...,None,None,2022-02-08,"""Three teenage boy cousins on a road trip thro...",BOOK,None,None,304.0


In [114]:
# Handle NaN values by replacing them with None (which MySQL interprets as NULL)
# leelow_sql_input = leelow_sql_input.where(pd.notnull(leelow_sql_input), None)

# Print the dataframe to check if NaNs are replaced by None
# print("Dataframe after replacing NaNs with None:")
# print(leelow_sql_input)

# # MySQL connection setup
# mydb = mysql.connector.connect(
#     host="localhost",
#     user="root",
#     database="Remo"
# )

# # Create a cursor object
# cursor = mydb.cursor()

# SQL INSERT statement
insert_stmt = """
    INSERT INTO LastTest (isbn, title, author, genre, subgenre, lexileLevel, publisher, copyrightDate, summary, type, form, seriesName/Position, pageCount)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# SQL SELECT statement
select_stmt = """
    SELECT COUNT(*) FROM LastTest WHERE isbn = %s
"""

# Loop through each row in the dataframe
for _, row in leelow_sql_input.iterrows():
    # Convert datetime to string if it's not None
    copyright_date_str = row['copyrightDate'].strftime('%Y-%m-%d') if row['copyrightDate'] else None

    # Print the values being inserted to debug
    print(f"Inserting values: {row['isbn']}, {row['title']}, {row['author']}, {row['genre']}, {row['subgenre']}, "
          f"{row['lexileLevel']}, {row['publisher']}, {copyright_date_str}, {row['summary']}, {row['type']}, "
          f"{row['form']}, {row['seriesName/Position']}, {row['pageCount']}")

    # Extract values from each row
    values = (
        row['isbn'],
        row['title'],
        row['author'],
        row['genre'],
        row['subgenre'],
        row['lexileLevel'],
        row['publisher'],
        copyright_date_str,  # Use formatted date string (or None if missing)
        row['summary'],
        row['type'],
        row['form'],
        row['seriesName/Position'],
        row['pageCount']
    )


    # Check if the record already exists by querying for the isbn
    cursor.execute(select_stmt, (row['isbn'],))
    result = cursor.fetchone()

    # If the record does not exist, insert it
    if result[0] == 0:  # result[0] contains the count from the SELECT query
        # Print the values being inserted to debug
        print(f"Inserting values: {row['isbn']}, {row['title']}, {row['author']}, {row['genre']}, "
              f"{row['subgenre']}, {row['lexileLevel']}, {row['publisher']}, {copyright_date_str}, {row['summary']}, "
              f"{row['type']}, {row['form']}, {row['seriesName/Publisher']}, {row['pageCount']}")

        # Execute the INSERT statement with the values
        cursor.execute(insert_stmt, values)
    else:
        # Print that the record is a duplicate and will not be inserted
        print(f"Duplicate found for ISBN {row['isbn']}, skipping insert.")

# Commit the transaction to save changes
mydb.commit()

# Close the cursor and the connection
cursor.close()
mydb.close()

print("Data processed successfully!")
#     # Execute the INSERT statement with the values
#     cursor.execute(insert_stmt, values)

# # Commit the transaction to save changes
# mydb.commit()

# # Close the cursor and the connection
# cursor.close()
# mydb.close()

# print("Data inserted successfully!")

Inserting values: 9781643790664, Miosotis Flores Never Forgets, Hilda Eunice Burgos, None, books for middle school girls;ages 9-12;rescue dogs;Dominican American;DR;Hispanic;Latinx;domestic violence;domestic abuse;LGBTQ;sisters;diverse middle grade;ownvoices;Meg Medina;grades 6-8;realistic fiction, None, None, 2021-10-26, None, None, None, None, None
Duplicate found for ISBN 9781643790664, skipping insert.
Inserting values: 9781643790657, Miosotis Flores Never Forgets, Hilda Eunice Burgos, None, books for middle school girls;ages 9-12;rescue dogs;Dominican American;DR;Hispanic;Latinx;domestic violence;domestic abuse;LGBTQ;sisters;diverse middle grade;ownvoices;Meg Medina;grades 6-8;realistic fiction, None, None, 2021-10-26, None, None, None, None, None
Duplicate found for ISBN 9781643790657, skipping insert.
Inserting values: 9781643794327, Black Was the Ink, Michelle Coles, Justin Johnson, None, Reconstruction;Black history;Black teens;ownvoices YA;Civil War;Klu Klux Klan;historical f

## Lerner Print xml

In [155]:
filename = '/Users/rwebb/Desktop/ReMo/ONIX/Lerner_Print_ONIX_20240104104306.xml'

# Parse the XML file
tree = ET.parse(filename)
root = tree.getroot()

# Create a list to store dictionaries representing each book
books_list = []

# Iterate through each <product> element
for product_elem in root.findall('.//product'):
    book_dict = {}
    
    # Extracting the ISBN from <a001> and <productidentifier> with <b221> 03
    isbn_1 = product_elem.find('a001').text if product_elem.find('a001') is not None else None
    isbn_2 = None
    for product_id_elem in product_elem.findall('productidentifier'):
        if product_id_elem.find('b221') is not None and product_id_elem.find('b221').text == '03':
            isbn_2 = product_id_elem.find('b244').text if product_id_elem.find('b244') is not None else None
    book_dict['ISBN'] = isbn_1
    book_dict['ISBN_Alternative'] = isbn_2
    
    # Extracting the Title from <titledetail> -> <b203>
    title = product_elem.find('.//titledetail//b203').text if product_elem.find('.//titledetail//b203') is not None else None
    book_dict['Title'] = title
    
    # Extracting Contributors (authors) from <contributor> -> <b036>
    contributors = []
    for contributor_elem in product_elem.findall('.//contributor'):
        name = contributor_elem.find('b036').text if contributor_elem.find('b036') is not None else None
        contributors.append(name)
    book_dict['Contributors'] = ', '.join(contributors)  # Join multiple names if there are multiple contributors
    
    # Extracting the Publisher from <publishingdetail> -> <b081>
    publisher = product_elem.find('.//publishingdetail//b081').text if product_elem.find('.//publishingdetail//b081') is not None else None
    book_dict['Publisher'] = publisher
    
    # Extracting the Price from <productsupply> -> <price> -> <j151>
    # price = product_elem.find('.//productsupply//price//j151').text if product_elem.find('.//productsupply//price//j151') is not None else None
    # book_dict['Price'] = price
    
    # Extracting the Activities (subject)
    subject_activities = []
    for subject_elem in product_elem.findall('.//subject'):
        subject_activity = subject_elem.find('b070').text if subject_elem.find('b070') is not None else None
        if subject_activity:
            subject_activities.append(subject_activity)
    book_dict['SubjectActivities'] = ', '.join(subject_activities)  # Join multiple subject activities if there are multiple
    
    # Extracting the Publishing Date (from <publishingdate> -> <b306>)
    publishing_date = product_elem.find('.//publishingdetail//publishingdate//b306').text if product_elem.find('.//publishingdetail//publishingdate//b306') is not None else None
    book_dict['PublishingDate'] = publishing_date

    # Append the dictionary to the list
    books_list.append(book_dict)

    # Extracting the Copyright Date (from <copyrightstatement> -> <b087>)
    copyright_date = product_elem.find('.//copyrightstatement//b087').text if product_elem.find('.//copyrightstatement//b087') is not None else None
    book_dict['CopyrightDate'] = copyright_date

# Create a Pandas DataFrame from the list of dictionaries
Lerner = pd.DataFrame(books_list)

# Print the resulting DataFrame and books_list for debugging
Lerner

,ISBN,ISBN_Alternative,Title,Contributors,Publisher,SubjectActivities,PublishingDate,CopyrightDate
0,9780761302971,9780761302971,The Winter Solstice,"Ellen Jackson, Jan Davey Ellis",Lerner Publishing Group,Activities; Changing of the Seasons; Craft-Mak...,20030801,1994
1,9780761314851,9780761314851,Jellies,Twig George,Lerner Publishing Group,Animals; Marine Life; Middle Grade; Middle Gra...,20000801,2000
2,9780761316473,9780761316473,Africa Is Not a Country,"Margy Burns Knight, Mark Melnicove, Anne Sible...",Lerner Publishing Group,Africa Is Not a Country; Africa; African Count...,20020801,2000
3,9780822505358,9780822505358,Picture Perfect,Elaine Marie Alphin,Lerner Publishing Group,Accusations; bereavement; Death Dying; Disappe...,20030801,2003
4,9780822531968,9780822531968,Animal Scavengers,Sandra Markle,Lerner Publishing Group,"JUVENILE NONFICTION / Animals / Insects, Spide...",20050801,2005
...,...,...,...,...,...,...,...,...
4203,9781916526969,9781916526969,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; US; World Hist...,20240801,2025
4204,9781916526976,9781916526976,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; world history;...,20240801,2025
4205,9781916526983,9781916526983,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; world history;...,20240801,2025
4206,9781916949300,9781916949300,Witness to War,Kelly Roberts,Cheriton Children's Books,army; encourages questioning; analyzing eviden...,20240801,2025


In [167]:
Lerner.rename(columns={'Title':'Title/Subtitle', 'Contributors':'Author'}, inplace=True)
add_attributes(Lerner)
Lerner

,ISBN,ISBN_Alternative,Title/Subtitle,Author,Publisher,SubjectActivities,PublishingDate,CopyrightDate,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type,Publication Year,Material Type,Subject,Lexile
0,9780761302971,9780761302971,The Winter Solstice,"Ellen Jackson, Jan Davey Ellis",Lerner Publishing Group,Activities; Changing of the Seasons; Craft-Mak...,20030801,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9780761314851,9780761314851,Jellies,Twig George,Lerner Publishing Group,Animals; Marine Life; Middle Grade; Middle Gra...,20000801,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9780761316473,9780761316473,Africa Is Not a Country,"Margy Burns Knight, Mark Melnicove, Anne Sible...",Lerner Publishing Group,Africa Is Not a Country; Africa; African Count...,20020801,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9780822505358,9780822505358,Picture Perfect,Elaine Marie Alphin,Lerner Publishing Group,Accusations; bereavement; Death Dying; Disappe...,20030801,2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9780822531968,9780822531968,Animal Scavengers,Sandra Markle,Lerner Publishing Group,"JUVENILE NONFICTION / Animals / Insects, Spide...",20050801,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4203,9781916526969,9781916526969,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; US; World Hist...,20240801,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4204,9781916526976,9781916526976,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; world history;...,20240801,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4205,9781916526983,9781916526983,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; world history;...,20240801,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4206,9781916949300,9781916949300,Witness to War,Kelly Roberts,Cheriton Children's Books,army; encourages questioning; analyzing eviden...,20240801,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
for isbn in Lerner['ISBN']:
    if pd.isna(isbn):
        continue  # Skip if ISBN is NaN
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    if response.status_code == 200: # Confirm request worked/is available
        book_data = response.json()
        if 'items' in book_data:
        # Extract book details from the API response
            book_info = book_data['items'][0]['volumeInfo']
            # Missing:
            # Series name and position
            # Form
            # Format
            book_details = {
                'ISBN#': isbn,
                'Title': book_info.get('title', np.nan),
                'Subtitle': np.nan, # Not available
                'Authors': ', '.join(book_info.get('authors', ['N/A'])),
                'Publisher': book_info.get('publisher', np.nan),
                'PublishedDate': book_info.get('publishedDate', np.nan),
                'CopyrightDate': np.nan, # Not available
                'Summary': book_info.get('description', 'No description available'), # NEED
                'Genre': ', '.join(book_info.get('categories', ['N/A'])), # NEED
                'PageCount': book_info.get('pageCount', np.nan), # NEED
                'Type': book_info.get('printType', np.nan), # Need
                'Categories': book_info.get('Categories', np.nan)
                # Type of book - ficiton, nonfiction, blended
            }
            # Accessing the correct row using ISBN in the DataFrame
            row_index = Lerner[Lerner['ISBN'] == isbn].index
            
            if not row_index.empty:  # Check if a matching ISBN was found
                row_index = row_index[0]  # Get the first (and expected only) match

                # Only update if the current value is NaN
                if pd.isna(Lerner.loc[row_index, 'Title/Subtitle']):
                    Lerner.loc[row_index, 'Title/Subtitle'] = book_details['Title']
                if pd.isna(Lerner.loc[row_index, 'Author']):
                    Lerner.loc[row_index, 'Author'] = book_details['Authors']
                if pd.isna(Lerner.loc[row_index, 'Publication Year']):
                    Lerner.loc[row_index, 'Publication Year'] = book_details['PublishedDate']
                if pd.isna(Lerner.loc[row_index, 'Publisher']):
                    Lerner.loc[row_index, 'Publisher'] = book_details['Publisher']
                if pd.isna(Lerner.loc[row_index, 'Material Type']):
                    Lerner.loc[row_index, 'Material Type'] = book_details['Type']
                if pd.isna(Lerner.loc[row_index, 'Subject']):
                    Lerner.loc[row_index, 'Subject'] = book_details['Genre']
                if pd.isna(Lerner.loc[row_index, 'Summary']):
                    Lerner.loc[row_index, 'Summary'] = book_details['Summary']
                if pd.isna(Lerner.loc[row_index, 'Page Count']):
                    Lerner.loc[row_index, 'Page Count'] = book_details['PageCount']

/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_82821/4238324225.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1994' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Lerner.loc[row_index, 'Publication Year'] = book_details['PublishedDate']
/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_82821/4238324225.py:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'BOOK' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  Lerner.loc[row_index, 'Material Type'] = book_details['Type']
/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_82821/4238324225.py:48: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Juvenile Nonfiction' has dtype inc

In [169]:
Lerner

,ISBN,ISBN_Alternative,Title/Subtitle,Author,Publisher,SubjectActivities,PublishingDate,CopyrightDate,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type,Publication Year,Material Type,Subject,Lexile
0,9780761302971,9780761302971,The Winter Solstice,"Ellen Jackson, Jan Davey Ellis",Lerner Publishing Group,Activities; Changing of the Seasons; Craft-Mak...,20030801,1994,NaN,Presents facts and folklore about the shortest...,NaN,NaN,NaN,NaN,36.0,NaN,1994,BOOK,Juvenile Nonfiction,NaN
1,9780761314851,9780761314851,Jellies,Twig George,Lerner Publishing Group,Animals; Marine Life; Middle Grade; Middle Gra...,20000801,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9780761316473,9780761316473,Africa Is Not a Country,"Margy Burns Knight, Mark Melnicove, Anne Sible...",Lerner Publishing Group,Africa Is Not a Country; Africa; African Count...,20020801,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9780822505358,9780822505358,Picture Perfect,Elaine Marie Alphin,Lerner Publishing Group,Accusations; bereavement; Death Dying; Disappe...,20030801,2003,NaN,A gap in his memory the afternoon that his bes...,NaN,NaN,NaN,NaN,256.0,NaN,2003-01-01,BOOK,Juvenile Fiction,NaN
4,9780822531968,9780822531968,Animal Scavengers,Sandra Markle,Lerner Publishing Group,"JUVENILE NONFICTION / Animals / Insects, Spide...",20050801,2005,NaN,Learn about the amazing world of army ants.,NaN,NaN,NaN,NaN,49.0,NaN,2005,BOOK,Juvenile Nonfiction,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4203,9781916526969,9781916526969,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; US; World Hist...,20240801,2025,NaN,The Vietnam War was one of the most divisive w...,NaN,NaN,NaN,NaN,0.0,NaN,2024-08,BOOK,Juvenile Nonfiction,NaN
4204,9781916526976,9781916526976,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; world history;...,20240801,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4205,9781916526983,9781916526983,Witness to War,Kelly Roberts,Cheriton Children's Books,military; army; social studies; world history;...,20240801,2025,NaN,World War II was one of the most wide-reaching...,NaN,NaN,NaN,NaN,0.0,NaN,2024-08,BOOK,Juvenile Nonfiction,NaN
4206,9781916949300,9781916949300,Witness to War,Kelly Roberts,Cheriton Children's Books,army; encourages questioning; analyzing eviden...,20240801,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## CRW Report xlsx

In [115]:
# filename = input('Enter File Name: ')
filename = '/Users/rwebb/Desktop/ReMo/CRWReportJob148737.xlsx'
print('Selected:', filename)

if filename.endswith('.mrc'):
    file = open(filename, 'r')
elif filename.endswith('.xml'):
    file = open(filename, 'r')
    # with file as xml_file:
    #     data_dict = xmltodict.parse(xml_file.read())
    #     file=pd.json_normalize(data_dict)
    tree = ET.parse(filename)
    root = tree.getroot()
    # Create a list to store dictionaries representing each book
    books_list = []
    # Iterate through each <book> element
    for book_elem in root.findall('.//Product'):
        book_dict = {}
        book_dict['RecordReference'] = book_elem.get('RecordReference')
        for child_elem in book_elem:
            book_dict[child_elem.tag] = child_elem.text
        books_list.append(book_dict)
    
    # Create a Pandas DataFrame from the list of dictionaries
    file = pd.DataFrame(books_list)
    print(books_list)
    # Display the DataFrame
    # xml_file.close()
    # df=pd.DataFrame()
    # file = pd.read_xml(filename)
    # file = pd.read_xml(filename, xpath='.//Product')
    
    # file = open(filename, 'r')
elif filename.endswith('.xlsx'):
    file = pd.read_excel(filename)
    # print(file.head())  # Uncomment if you want to inspect the first few rows
    isbn_values = []
    
    # Ensure 'ISBN' column exists
    if 'ISBN' in file.columns:
        for isbn in file['ISBN']:
            if pd.isna(isbn):
                continue  # Skip if ISBN is NaN
            isbn = str(isbn)  # Ensure it's a string
            isbn = isbn.replace('-', '')  # Remove dashes
            # Check if the ISBN starts with '978' or '979'
            if not (isbn[:3] == '978' or isbn[:3] == '979'):
                if len(isbn) == 10:
                    isbn = '978' + isbn
                else:    
                    isbn_values.append('BAD DATA')
                continue  # Skip the current invalid ISBN
            isbn = isbn[:13]  # Ensure it's the correct length (13 digits)

            isbn_values.append(isbn)
        # print(isbn_values)
    else:
        print("No 'ISBN' column found in the Excel file.")
else:
    print("Unsupported file format.")
crw_report = file

Selected: /Users/rwebb/Desktop/ReMo/CRWReportJob148737.xlsx


In [116]:
crw_report

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject
0,NaN,NaN,The 60s reader,NaN,"Haskins, James.",1987.0,"Viking Kestrel,",Book,NaN,United States -- History.
1,NaN,NaN,200 years in Union : a pictorial history of Un...,NaN,"Nash, Chester",1974.0,"Union Historical Society,",Book,NaN,Union (Me.) -- History -- Pictoral works.
2,NaN,NaN,"1929, the year of the great crash",NaN,"Klingaman, William K.",1989.0,"Harper & Row,",Book,NaN,Depressions -- 1929 -- United States.
3,NaN,NaN,"1929, the year of the great crash",NaN,"Klingaman, William K.",1989.0,"Harper & Row,",Book,NaN,Economic history -- 1918-1945.
4,NaN,NaN,"1929, the year of the great crash",NaN,"Klingaman, William K.",1989.0,"Harper & Row,",Book,NaN,United States -- Economic conditions -- 1918-1...
...,...,...,...,...,...,...,...,...,...,...
79802,979-8-98528280-1,NaN,Women are the fiercest creatures,NaN,"Dunlop, Andrea.",2023.0,"Zibby Books,",Book,NaN,Families -- Fiction.
79803,979-8-98528280-1,NaN,Women are the fiercest creatures,NaN,"Dunlop, Andrea.",2023.0,"Zibby Books,",Book,NaN,\tFeminists -- Fiction.
79804,979-8-98528280-1,NaN,Women are the fiercest creatures,NaN,"Dunlop, Andrea.",2023.0,"Zibby Books,",Book,NaN,Friendship -- Women -- Fiction.
79805,979-8-98528280-1,NaN,Women are the fiercest creatures,NaN,"Dunlop, Andrea.",2023.0,"Zibby Books,",Book,NaN,Thriller fiction.


In [117]:
nan_df = crw_report[crw_report['ISBN'].isna()]
isbn_df = crw_report[~crw_report['ISBN'].isna()]

In [118]:
isbn_df.loc[:, 'ISBN'] = isbn_df['ISBN'].str.replace('-', '', regex=False)
isbn_df.loc[:, 'ISBN'] = isbn_df['ISBN'].apply(modify_isbn)

In [119]:
broken_isbn_df = isbn_df[~isbn_df['ISBN'].str[:3].isin(['978', '979'])]
whole_isbn_df = isbn_df[isbn_df['ISBN'].str[:3].isin(['978', '979'])]

broken_isbn_df

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject
947,0020345542 (p,NaN,Great battles of the Civil War,NaN,"Macdonald, John, 1945-",1992.0,"Collier/Macmillan,",Book,NaN,United States -- History -- 1861-1865.
952,0020518609 (p,Scribner classics,The short stories of Ernest Hemingway,NaN,"Hemingway, Ernest, 1899-1961.",1986.0,"Collier Books,",Book,NaN,Short stories.
980,0030705061 (p,NaN,Paper flight : 48 models ready for take-off,NaN,"Botermans, Jack.",1984.0,"Holt,",Book,NaN,Paper airplanes.
1218,0060248742 (l,NaN,Leaving home : stories,NaN,selected by Hazel Rochman and Darlene Z. McCam...,1997.0,"HarperCollins Publishers,",Book,NaN,Literature -- Collections.
1219,0060248742 (l,NaN,Leaving home : stories,NaN,selected by Hazel Rochman and Darlene Z. McCam...,1997.0,"HarperCollins Publishers,",Book,NaN,Young adult literature.
...,...,...,...,...,...,...,...,...,...,...
76180,1887178228 (a,NaN,A world lost,NaN,"Berry, Wendell, 1934-",1996.0,Counterpoint ;,Book,NaN,Kentucky -- Fiction.
76181,1887178228 (a,NaN,A world lost,NaN,"Berry, Wendell, 1934-",1996.0,Counterpoint ;,Book,NaN,Uncles -- Fiction.
79642,3822829668 (p,Basic art series.,Futurism,NaN,"Martin, Sylvia, 1964-",2005.0,"Taschen,",Book,NaN,Futurism (Art)
79646,3822852961 (p,NaN,Renaissance,NaN,"Wundram, Manfred.",2006.0,"Taschen,",Book,NaN,"Painting, Renaissance."


In [120]:
broken_isbn_df.loc[:,'ISBN'] = broken_isbn_df['ISBN'].apply(lambda x: '978' + str(x)[:10]) # Is 978 the right choice here??
broken_isbn_df

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject
947,9780020345542,NaN,Great battles of the Civil War,NaN,"Macdonald, John, 1945-",1992.0,"Collier/Macmillan,",Book,NaN,United States -- History -- 1861-1865.
952,9780020518609,Scribner classics,The short stories of Ernest Hemingway,NaN,"Hemingway, Ernest, 1899-1961.",1986.0,"Collier Books,",Book,NaN,Short stories.
980,9780030705061,NaN,Paper flight : 48 models ready for take-off,NaN,"Botermans, Jack.",1984.0,"Holt,",Book,NaN,Paper airplanes.
1218,9780060248742,NaN,Leaving home : stories,NaN,selected by Hazel Rochman and Darlene Z. McCam...,1997.0,"HarperCollins Publishers,",Book,NaN,Literature -- Collections.
1219,9780060248742,NaN,Leaving home : stories,NaN,selected by Hazel Rochman and Darlene Z. McCam...,1997.0,"HarperCollins Publishers,",Book,NaN,Young adult literature.
...,...,...,...,...,...,...,...,...,...,...
76180,9781887178228,NaN,A world lost,NaN,"Berry, Wendell, 1934-",1996.0,Counterpoint ;,Book,NaN,Kentucky -- Fiction.
76181,9781887178228,NaN,A world lost,NaN,"Berry, Wendell, 1934-",1996.0,Counterpoint ;,Book,NaN,Uncles -- Fiction.
79642,9783822829668,Basic art series.,Futurism,NaN,"Martin, Sylvia, 1964-",2005.0,"Taschen,",Book,NaN,Futurism (Art)
79646,9783822852961,NaN,Renaissance,NaN,"Wundram, Manfred.",2006.0,"Taschen,",Book,NaN,"Painting, Renaissance."


In [121]:
isbn_df = pd.concat([broken_isbn_df, whole_isbn_df])
isbn_df

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject
947,9780020345542,NaN,Great battles of the Civil War,NaN,"Macdonald, John, 1945-",1992.0,"Collier/Macmillan,",Book,NaN,United States -- History -- 1861-1865.
952,9780020518609,Scribner classics,The short stories of Ernest Hemingway,NaN,"Hemingway, Ernest, 1899-1961.",1986.0,"Collier Books,",Book,NaN,Short stories.
980,9780030705061,NaN,Paper flight : 48 models ready for take-off,NaN,"Botermans, Jack.",1984.0,"Holt,",Book,NaN,Paper airplanes.
1218,9780060248742,NaN,Leaving home : stories,NaN,selected by Hazel Rochman and Darlene Z. McCam...,1997.0,"HarperCollins Publishers,",Book,NaN,Literature -- Collections.
1219,9780060248742,NaN,Leaving home : stories,NaN,selected by Hazel Rochman and Darlene Z. McCam...,1997.0,"HarperCollins Publishers,",Book,NaN,Young adult literature.
...,...,...,...,...,...,...,...,...,...,...
79802,9798985282801,NaN,Women are the fiercest creatures,NaN,"Dunlop, Andrea.",2023.0,"Zibby Books,",Book,NaN,Families -- Fiction.
79803,9798985282801,NaN,Women are the fiercest creatures,NaN,"Dunlop, Andrea.",2023.0,"Zibby Books,",Book,NaN,\tFeminists -- Fiction.
79804,9798985282801,NaN,Women are the fiercest creatures,NaN,"Dunlop, Andrea.",2023.0,"Zibby Books,",Book,NaN,Friendship -- Women -- Fiction.
79805,9798985282801,NaN,Women are the fiercest creatures,NaN,"Dunlop, Andrea.",2023.0,"Zibby Books,",Book,NaN,Thriller fiction.


In [122]:
isbn_df = isbn_df.astype(str)

In [123]:
# Drop duplicates based on relevant columns while keeping the first occurrence
isbn_df_unique = isbn_df.drop_duplicates(subset=['ISBN', 'Series Title', 'Title/Subtitle', 'ISSN', 
                                                  'Author', 'Publication Year', 'Publisher', 
                                                  'Material Type', 'Lexile'], keep='first')

# Group by 'ISBN' and combine the 'Subject' values
isbn_df_unique = isbn_df_unique.groupby('ISBN').agg({
    'Series Title': 'first',  # Keep the first value (they are the same for each ISBN)
    'Title/Subtitle': 'first',  # Same as above
    'ISSN': 'first',  # Same as above
    'Author': 'first',  # Same as above
    'Publication Year': 'first',  # Same as above
    'Publisher': 'first',  # Same as above
    'Material Type': 'first',  # Same as above
    'Lexile': 'first',  # Same as above
    'Subject': ', '.join  # Combine all unique 'Subject' values
}).reset_index()

isbn_df_unique

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject
0,9780007439126,Martin Beck series,The Man Who Went Up in Smoke,nan,"Sjöwall, Maj.",2016.0,"Fourth Estate,",Book,nan,"Beck, Martin (Fictitious character)"
1,9780007511242,nan,It's about love,nan,"Camden, Steven.",2015.0,"HarperCollins Children's Books,",Book,nan,Families
2,9780008183684,nan,Scotland the best 100 places,nan,"Irvine, Peter, 1948-",2017.0,"Collins,",Book,nan,Nature photography.
3,9780008256944,nan,Lateral thinking puzzlers,nan,Paul Sloane.,2016.0,"Puzzlewright Press, an imprint of Sterling Pub...",Book,nan,Lateral thinking.
4,9780020198817,A Scribner classic,The great Gatsby,nan,"Fitzgerald, F. Scott (Francis Scott), 1896-1940.",1992.0,"Collier Books,",Book,1070L,nan
...,...,...,...,...,...,...,...,...,...,...
11286,9789380741222,Campfire heroes,Gandhi : my life is my message,nan,"Quinn, Jason.",2013.0,"Campifre/Kalyani Navyug Media PVT Ltd.,",Book,nan,"Cartoons and comics., Cartoons and comics."
11287,9789629549992,nan,The great Gatsby,nan,"Fitzgerald, F. Scott (Francis Scott), 1896-1940.",2012.0,"Findaway World,",Sound,nan,Audiobooks.
11288,9798639873775,nan,The fifth vital,nan,"Majlak, Mike, 1985-",2020.0,"Mike Majlak,",Book,nan,Autobiographies.
11289,9798663066952,nan,Schitt's Creek Fun Facts : 80 fun facts and ev...,nan,"Ortiz, Celestina.",2020.0,"Independently Published,",Book,nan,American culture.


In [124]:
add_attributes(isbn_df_unique)
add_attributes(nan_df)
isbn_df_unique

/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_1985/3954768506.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,category] = np.nan
/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_1985/3954768506.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,category] = np.nan
/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_1985/3954768506.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type
0,9780007439126,Martin Beck series,The Man Who Went Up in Smoke,nan,"Sjöwall, Maj.",2016.0,"Fourth Estate,",Book,nan,"Beck, Martin (Fictitious character)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9780007511242,nan,It's about love,nan,"Camden, Steven.",2015.0,"HarperCollins Children's Books,",Book,nan,Families,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9780008183684,nan,Scotland the best 100 places,nan,"Irvine, Peter, 1948-",2017.0,"Collins,",Book,nan,Nature photography.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9780008256944,nan,Lateral thinking puzzlers,nan,Paul Sloane.,2016.0,"Puzzlewright Press, an imprint of Sterling Pub...",Book,nan,Lateral thinking.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9780020198817,A Scribner classic,The great Gatsby,nan,"Fitzgerald, F. Scott (Francis Scott), 1896-1940.",1992.0,"Collier Books,",Book,1070L,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11286,9789380741222,Campfire heroes,Gandhi : my life is my message,nan,"Quinn, Jason.",2013.0,"Campifre/Kalyani Navyug Media PVT Ltd.,",Book,nan,"Cartoons and comics., Cartoons and comics.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11287,9789629549992,nan,The great Gatsby,nan,"Fitzgerald, F. Scott (Francis Scott), 1896-1940.",2012.0,"Findaway World,",Sound,nan,Audiobooks.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11288,9798639873775,nan,The fifth vital,nan,"Majlak, Mike, 1985-",2020.0,"Mike Majlak,",Book,nan,Autobiographies.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11289,9798663066952,nan,Schitt's Creek Fun Facts : 80 fun facts and ev...,nan,"Ortiz, Celestina.",2020.0,"Independently Published,",Book,nan,American culture.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
for isbn in isbn_df_unique['ISBN']:
    if pd.isna(isbn):
        continue  # Skip if ISBN is NaN
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    if response.status_code == 200: # Confirm request worked/is available
        book_data = response.json()
        if 'items' in book_data:
        # Extract book details from the API response
            book_info = book_data['items'][0]['volumeInfo']
            # Missing:
            # Series name and position
            # Form
            # Format
            book_details = {
                'ISBN#': isbn,
                'Title': book_info.get('title', np.nan),
                'Subtitle': np.nan, # Not available
                'Authors': ', '.join(book_info.get('authors', ['N/A'])),
                'Publisher': book_info.get('publisher', np.nan),
                'PublishedDate': book_info.get('publishedDate', np.nan),
                'CopyrightDate': np.nan, # Not available
                'Summary': book_info.get('description', 'No description available'), # NEED
                'Genre': ', '.join(book_info.get('categories', ['N/A'])), # NEED
                'PageCount': book_info.get('pageCount', np.nan), # NEED
                'Type': book_info.get('printType', np.nan), # Need
                'Categories': book_info.get('Categories', np.nan)
                # Type of book - ficiton, nonfiction, blended
            }
            # Accessing the correct row using ISBN in the DataFrame
            row_index = isbn_df_unique[isbn_df_unique['ISBN'] == isbn].index
            
            if not row_index.empty:  # Check if a matching ISBN was found
                row_index = row_index[0]  # Get the first (and expected only) match

                # Only update if the current value is NaN
                if pd.isna(isbn_df_unique.loc[row_index, 'Title/Subtitle']):
                    isbn_df_unique.loc[row_index, 'Title/Subtitle'] = book_details['Title']
                if pd.isna(isbn_df_unique.loc[row_index, 'Author']):
                    isbn_df_unique.loc[row_index, 'Author'] = book_details['Authors']
                if pd.isna(isbn_df_unique.loc[row_index, 'Publication Year']):
                    isbn_df_unique.loc[row_index, 'Publication Year'] = book_details['PublishedDate']
                if pd.isna(isbn_df_unique.loc[row_index, 'Publisher']):
                    isbn_df_unique.loc[row_index, 'Publisher'] = book_details['Publisher']
                if pd.isna(isbn_df_unique.loc[row_index, 'Material Type']):
                    isbn_df_unique.loc[row_index, 'Material Type'] = book_details['Type']
                if pd.isna(isbn_df_unique.loc[row_index, 'Subject']):
                    isbn_df_unique.loc[row_index, 'Subject'] = book_details['Genre']
                if pd.isna(isbn_df_unique.loc[row_index, 'Summary']):
                    isbn_df_unique.loc[row_index, 'Summary'] = book_details['Summary']
                if pd.isna(isbn_df_unique.loc[row_index, 'Page Count']):
                    isbn_df_unique.loc[row_index, 'Page Count'] = book_details['PageCount']

/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_1985/3857278171.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Detective Martin Beck has just begun his holiday on a small island off the coast of Sweden. But when a neighbour gets a phone call, Beck finds himself packed off to Budapest, where a boorish journalist has vanished without a trace.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  isbn_df_unique.loc[row_index, 'Summary'] = book_details['Summary']


In [127]:
isbn_df_unique

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type
0,9780007439126,Martin Beck series,The Man Who Went Up in Smoke,nan,"Sjöwall, Maj.",2016.0,"Fourth Estate,",Book,nan,"Beck, Martin (Fictitious character)",NaN,Detective Martin Beck has just begun his holid...,NaN,NaN,NaN,NaN,0.0,NaN
1,9780007511242,nan,It's about love,nan,"Camden, Steven.",2015.0,"HarperCollins Children's Books,",Book,nan,Families,NaN,"Despite coming from different backgrounds, asp...",NaN,NaN,NaN,NaN,0.0,NaN
2,9780008183684,nan,Scotland the best 100 places,nan,"Irvine, Peter, 1948-",2017.0,"Collins,",Book,nan,Nature photography.,NaN,"Peter Irvine, bestselling author of Scotland t...",NaN,NaN,NaN,NaN,0.0,NaN
3,9780008256944,nan,Lateral thinking puzzlers,nan,Paul Sloane.,2016.0,"Puzzlewright Press, an imprint of Sterling Pub...",Book,nan,Lateral thinking.,NaN,"The astonishing, beautiful new story for all r...",NaN,NaN,NaN,NaN,400.0,NaN
4,9780020198817,A Scribner classic,The great Gatsby,nan,"Fitzgerald, F. Scott (Francis Scott), 1896-1940.",1992.0,"Collier Books,",Book,1070L,nan,NaN,"Jay Gatsby had once loved beautiful, spoiled D...",NaN,NaN,NaN,NaN,248.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11286,9789380741222,Campfire heroes,Gandhi : my life is my message,nan,"Quinn, Jason.",2013.0,"Campifre/Kalyani Navyug Media PVT Ltd.,",Book,nan,"Cartoons and comics., Cartoons and comics.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11287,9789629549992,nan,The great Gatsby,nan,"Fitzgerald, F. Scott (Francis Scott), 1896-1940.",2012.0,"Findaway World,",Sound,nan,Audiobooks.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11288,9798639873775,nan,The fifth vital,nan,"Majlak, Mike, 1985-",2020.0,"Mike Majlak,",Book,nan,Autobiographies.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11289,9798663066952,nan,Schitt's Creek Fun Facts : 80 fun facts and ev...,nan,"Ortiz, Celestina.",2020.0,"Independently Published,",Book,nan,American culture.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
complete_df = pd.concat([isbn_df_unique, nan_df])

In [129]:
complete_df

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type
0,9780007439126,Martin Beck series,The Man Who Went Up in Smoke,nan,"Sjöwall, Maj.",2016.0,"Fourth Estate,",Book,nan,"Beck, Martin (Fictitious character)",NaN,Detective Martin Beck has just begun his holid...,NaN,NaN,NaN,NaN,0.0,NaN
1,9780007511242,nan,It's about love,nan,"Camden, Steven.",2015.0,"HarperCollins Children's Books,",Book,nan,Families,NaN,"Despite coming from different backgrounds, asp...",NaN,NaN,NaN,NaN,0.0,NaN
2,9780008183684,nan,Scotland the best 100 places,nan,"Irvine, Peter, 1948-",2017.0,"Collins,",Book,nan,Nature photography.,NaN,"Peter Irvine, bestselling author of Scotland t...",NaN,NaN,NaN,NaN,0.0,NaN
3,9780008256944,nan,Lateral thinking puzzlers,nan,Paul Sloane.,2016.0,"Puzzlewright Press, an imprint of Sterling Pub...",Book,nan,Lateral thinking.,NaN,"The astonishing, beautiful new story for all r...",NaN,NaN,NaN,NaN,400.0,NaN
4,9780020198817,A Scribner classic,The great Gatsby,nan,"Fitzgerald, F. Scott (Francis Scott), 1896-1940.",1992.0,"Collier Books,",Book,1070L,nan,NaN,"Jay Gatsby had once loved beautiful, spoiled D...",NaN,NaN,NaN,NaN,248.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912,NaN,Vintage classics.,Herman Melville's Moby Dick,NaN,United Artists ; Warner Bros. presents A Mouli...,2001.0,"MGM Home Entertainment,",Video,NaN,Action and adventure films.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
913,NaN,Vintage classics.,Herman Melville's Moby Dick,NaN,United Artists ; Warner Bros. presents A Mouli...,2001.0,"MGM Home Entertainment,",Video,NaN,Feature films.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
914,NaN,Vintage classics.,Herman Melville's Moby Dick,NaN,United Artists ; Warner Bros. presents A Mouli...,2001.0,"MGM Home Entertainment,",Video,NaN,Video recordings for the hearing impaired.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
915,NaN,Vintage classics.,Herman Melville's Moby Dick,NaN,United Artists ; Warner Bros. presents A Mouli...,2001.0,"MGM Home Entertainment,",Video,NaN,Whaling -- Drama.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Library Title Copy Report

In [185]:
# /Users/rwebb/Desktop/ReMo/LibraryTitleCopyReportJob148738.xlsx

In [253]:
# filename = input('Enter File Name: ')
filename = '/Users/rwebb/Desktop/ReMo/LibraryTitleCopyReportJob148738.xlsx'
print('Selected:', filename)

if filename.endswith('.mrc'):
    file = open(filename, 'r')
elif filename.endswith('.xml'):
    file = open(filename, 'r')
    # with file as xml_file:
    #     data_dict = xmltodict.parse(xml_file.read())
    #     file=pd.json_normalize(data_dict)
    tree = ET.parse(filename)
    root = tree.getroot()
    # Create a list to store dictionaries representing each book
    books_list = []
    # Iterate through each <book> element
    for book_elem in root.findall('.//Product'):
        book_dict = {}
        book_dict['RecordReference'] = book_elem.get('RecordReference')
        for child_elem in book_elem:
            book_dict[child_elem.tag] = child_elem.text
        books_list.append(book_dict)
    
    # Create a Pandas DataFrame from the list of dictionaries
    file = pd.DataFrame(books_list)
    print(books_list)
    # Display the DataFrame
    # xml_file.close()
    # df=pd.DataFrame()
    # file = pd.read_xml(filename)
    # file = pd.read_xml(filename, xpath='.//Product')
    
    # file = open(filename, 'r')
elif filename.endswith('.xlsx'):
    file = pd.read_excel(filename)
    # print(file.head())  # Uncomment if you want to inspect the first few rows
    isbn_values = []
    
    # Ensure 'ISBN' column exists
    if 'ISBN' in file.columns:
        for isbn in file['ISBN']:
            if pd.isna(isbn):
                continue  # Skip if ISBN is NaN
            isbn = str(isbn)  # Ensure it's a string
            isbn = isbn.replace('-', '')  # Remove dashes
            # Check if the ISBN starts with '978' or '979'
            if not (isbn[:3] == '978' or isbn[:3] == '979'):
                if len(isbn) == 10:
                    isbn = '978' + isbn
                else:    
                    isbn_values.append('BAD DATA')
                continue  # Skip the current invalid ISBN
            isbn = isbn[:13]  # Ensure it's the correct length (13 digits)

            isbn_values.append(isbn)
        # print(isbn_values)
    else:
        print("No 'ISBN' column found in the Excel file.")
else:
    print("Unsupported file format.")
library_title = file

Selected: /Users/rwebb/Desktop/ReMo/LibraryTitleCopyReportJob148738.xlsx
No 'ISBN' column found in the Excel file.


In [254]:
library_title

,Call Number,Title,Material Type,Author,Standard Number,LCCN,Total Copies,Copies Available,Copies Checked Out,Copies On Hold,Copies Loaned Out,Copies On Order,Copies Out For Repairs,Copies In Transit,Copies Lost
0,NaN,Banana Wars.,Book,NaN,NaN,NaN,1,0,0,0,0,0,0,0,1
1,NaN,The Cursed Sea.,Book,NaN,NaN,NaN,1,0,0,0,0,0,0,0,1
2,NaN,National geographic index.,Serial,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0
3,NaN,One of Us Is Lying.,Book,NaN,NaN,NaN,1,1,0,0,0,0,0,0,0
4,NaN,Science year; the World Book science annual,Book,NaN,ISSN: 0080-7621,LCCN: 65-21776,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10076,VID 970.1 WAB,Wabanaki a new dawn,Video,presented by the Maine Indian Tribal-State Com...,NaN,NaN,1,0,0,0,0,0,0,0,1
10077,VID 973.9 AME,The American experience. America 1900,Video,produced and directed by David Grubin ; writte...,ISBN: 0-7806-2412-2,NaN,2,0,0,0,0,0,0,0,2
10078,VID F COL,The color purple,Video,Warner Brothers.,ISBN: 0-7907-2971-7,NaN,1,0,0,0,0,0,0,0,1
10079,VID F PRI,Pride & prejudice,Video,Working Title Films ; Studio Canal ; produced ...,ISBN: 1-41705506-5,NaN,1,0,0,0,0,0,0,0,1


In [255]:
# Create the ISBN column and copy values where condition is met
library_title['ISBN'] = np.nan
library_title.loc[library_title['Standard Number'].str.startswith('ISBN', na=False), 'ISBN'] = library_title['Standard Number']

/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_82821/3181743657.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['ISBN: 0-7936-9327-6' 'ISBN: 1-64126-055-6'
 'ISBN: 978-0-452-28295-7 (pbk.)' ... 'ISBN: 0-7806-2412-2'
 'ISBN: 0-7907-2971-7' 'ISBN: 1-41705506-5']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  library_title.loc[library_title['Standard Number'].str.startswith('ISBN', na=False), 'ISBN'] = library_title['Standard Number']


In [256]:
library_title.drop(columns=['Call Number', 'Standard Number', 'LCCN', 'Total Copies', 'Copies Available', 'Copies Checked Out',
                            'Copies On Hold', 'Copies Loaned Out', 'Copies On Order', 'Copies Out For Repairs', 'Copies In Transit',
                            'Copies Lost'], inplace=True)

In [257]:
library_title

,Title,Material Type,Author,ISBN
0,Banana Wars.,Book,NaN,NaN
1,The Cursed Sea.,Book,NaN,NaN
2,National geographic index.,Serial,NaN,NaN
3,One of Us Is Lying.,Book,NaN,NaN
4,Science year; the World Book science annual,Book,NaN,NaN
...,...,...,...,...
10076,Wabanaki a new dawn,Video,presented by the Maine Indian Tribal-State Com...,NaN
10077,The American experience. America 1900,Video,produced and directed by David Grubin ; writte...,ISBN: 0-7806-2412-2
10078,The color purple,Video,Warner Brothers.,ISBN: 0-7907-2971-7
10079,Pride & prejudice,Video,Working Title Films ; Studio Canal ; produced ...,ISBN: 1-41705506-5


In [258]:
add_attributes(library_title)

In [259]:
library_title

,Title,Material Type,Author,ISBN,Title/Subtitle,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type,Publisher,Publication Year,Subject,Lexile
0,Banana Wars.,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Cursed Sea.,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,National geographic index.,Serial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,One of Us Is Lying.,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Science year; the World Book science annual,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10076,Wabanaki a new dawn,Video,presented by the Maine Indian Tribal-State Com...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10077,The American experience. America 1900,Video,produced and directed by David Grubin ; writte...,ISBN: 0-7806-2412-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10078,The color purple,Video,Warner Brothers.,ISBN: 0-7907-2971-7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10079,Pride & prejudice,Video,Working Title Films ; Studio Canal ; produced ...,ISBN: 1-41705506-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
def convert_isbn(isbn):
    if pd.isna(isbn):
        return np.nan
    
    # Remove the "ISBN: " prefix and hyphens
    str(isbn)
    if isbn[0:4] == 'ISBN':
        isbn = isbn.replace('ISBN: ', '').replace('-', '')
    
    if (isbn[:3] != '978' and isbn[:3] != '979'):
        isbn = '978' + isbn
    # Create the 978 and 979 variations
    # isbn_978 = '978' + isbn
    # isbn_979 = '979' + isbn
    
    # Return the ISBNs as a comma-separated string
    return isbn[0:13]

In [277]:
library_title['ISBN'] = library_title['ISBN'].apply(convert_isbn)
library_title.head(50)

,Title,Material Type,Author,ISBN,Title/Subtitle,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type,Publisher,Publication Year,Subject,Lexile
0,Banana Wars.,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Cursed Sea.,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,National geographic index.,Serial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,One of Us Is Lying.,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Science year; the World Book science annual,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,White Trash - The 400 Year Untold Story of Cla...,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,The Wild Dyer.,Book,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Crucible of empire the Spanish-American War,Video,"a production of Great Projects Film Company, I...",9780793693276,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Coloring Books for Teens : Ocean Designs,Book,Art.,9781641260556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"The atomic bomb; the great decision,",Book,"Baker, Paul R.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [278]:
for isbn in library_title['ISBN']:
    if pd.isna(isbn):
        continue  # Skip if ISBN is NaN
    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
    response = requests.get(url)
    if response.status_code == 200: # Confirm request worked/is available
        book_data = response.json()
        if 'items' in book_data:
        # Extract book details from the API response
            book_info = book_data['items'][0]['volumeInfo']
            # Missing:
            # Series name and position
            # Form
            # Format
            book_details = {
                'ISBN#': isbn,
                'Title': book_info.get('title', np.nan),
                'Subtitle': np.nan, # Not available
                'Authors': ', '.join(book_info.get('authors', ['N/A'])),
                'Publisher': book_info.get('publisher', np.nan),
                'PublishedDate': book_info.get('publishedDate', np.nan),
                'CopyrightDate': np.nan, # Not available
                'Summary': book_info.get('description', 'No description available'), # NEED
                'Genre': ', '.join(book_info.get('categories', ['N/A'])), # NEED
                'PageCount': book_info.get('pageCount', np.nan), # NEED
                'Type': book_info.get('printType', np.nan), # Need
                'Categories': book_info.get('Categories', np.nan)
                # Type of book - ficiton, nonfiction, blended
            }
            # Accessing the correct row using ISBN in the DataFrame
            row_index = library_title[library_title['ISBN'] == isbn].index
            
            if not row_index.empty:  # Check if a matching ISBN was found
                row_index = row_index[0]  # Get the first (and expected only) match

                # Only update if the current value is NaN
                if pd.isna(library_title.loc[row_index, 'Title/Subtitle']):
                    library_title.loc[row_index, 'Title/Subtitle'] = book_details['Title']
                if pd.isna(library_title.loc[row_index, 'Author']):
                    library_title.loc[row_index, 'Author'] = book_details['Authors']
                if pd.isna(library_title.loc[row_index, 'Publication Year']):
                    library_title.loc[row_index, 'Publication Year'] = book_details['PublishedDate']
                if pd.isna(library_title.loc[row_index, 'Publisher']):
                    library_title.loc[row_index, 'Publisher'] = book_details['Publisher']
                if pd.isna(library_title.loc[row_index, 'Material Type']):
                    library_title.loc[row_index, 'Material Type'] = book_details['Type']
                if pd.isna(library_title.loc[row_index, 'Subject']):
                    library_title.loc[row_index, 'Subject'] = book_details['Genre']
                if pd.isna(library_title.loc[row_index, 'Summary']):
                    library_title.loc[row_index, 'Summary'] = book_details['Summary']
                if pd.isna(library_title.loc[row_index, 'Page Count']):
                    library_title.loc[row_index, 'Page Count'] = book_details['PageCount']

In [280]:
library_title.tail(50)

,Title,Material Type,Author,ISBN,Title/Subtitle,Copyright Date,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type,Publisher,Publication Year,Subject,Lexile
10031,Freakboy,Book,"Clark, Kristin Elizabeth.",9781250062956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10032,Moonrise,Book,"Crossan, Sarah.",9781681193663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10033,One,Book,"Crossan, Sarah.",9780062118752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10034,The name she gave me,Book,"Culley, Betty.",9780063157835,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10035,Blood moon,Book,"Cuthew, Lucy.",9781536215038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10036,Bull,Book,"Elliott, David, 1947-",9781328596338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10037,Keesha's house,Book,"Frost, Helen, 1949-",9780374340641,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10038,A place like this,Book,"Herrick, Steven.",9780689867115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10039,Out of the dust,Book,"Hesse, Karen.",9780590371258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10040,Witness,Book,"Hesse, Karen.",9781725418219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [281]:
# Handle 978 vs 979

## MRC

In [ ]:
# Converted .mrc file to csv using Zotero

In [113]:
blueHill = pd.read_csv('/Users/rwebb/Desktop/ReMo/BlueHillConsolidatedSchool.csv')
blueHill

/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_82821/3981637368.py:1: DtypeWarning: Columns (9,10,22,23,27,35,39,44) have mixed types. Specify dtype option on import or set low_memory=False.
  blueHill = pd.read_csv('/Users/rwebb/Desktop/ReMo/BlueHillConsolidatedSchool.csv')


,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,...,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,BBF95TEY,book,NaN,Author,Title,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HC4M3PQB,book,2003.0,"Watts, Barrie",Butterfly,NaN,1-58340-234-9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,YQQ39KTF,book,2003.0,"Watts, Barrie",Sunflower,NaN,1-58340-232-2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HDF6WZIW,book,2003.0,"Watts, Barrie",Mouse,NaN,1-58340-231-4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BV6WGJJK,book,2002.0,"Watts, Barrie",Snake,NaN,1-58340-200-4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11288,RW5ZL2CB,book,2022.0,"Knox, Kelly",Be more obi-wan: Navigate your world with wit ...,NaN,978-0-7440-7772-8,NaN,NaN,http://link.overdrive.com/?websiteID=202788&ti...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11289,2KEH6KYZ,book,2021.0,"Horton, Cole",Star wars: battles that changed the galaxy,NaN,978-0-7440-5740-9,NaN,NaN,http://link.overdrive.com/?websiteID=202788&ti...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11290,QEYQL3MM,book,2007.0,"Piven, |Hanokh",My dog is as smelly as dirty socks: and other ...,NaN,978-0-375-84052-4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11291,GD5RV9GW,book,2009.0,"Hahn, Mary Downing",Closed for the season: a mystery story,NaN,978-0-547-39853-2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
blueHill.columns

Index(['Key', 'Item Type', 'Publication Year', 'Author', 'Title',
       'Publication Title', 'ISBN', 'ISSN', 'DOI', 'Url', 'Abstract Note',
       'Date', 'Date Added', 'Date Modified', 'Access Date', 'Pages',
       'Num Pages', 'Issue', 'Volume', 'Number Of Volumes',
       'Journal Abbreviation', 'Short Title', 'Series', 'Series Number',
       'Series Text', 'Series Title', 'Publisher', 'Place', 'Language',
       'Rights', 'Type', 'Archive', 'Archive Location', 'Library Catalog',
       'Call Number', 'Extra', 'Notes', 'File Attachments', 'Link Attachments',
       'Manual Tags', 'Automatic Tags', 'Editor', 'Series Editor',
       'Translator', 'Contributor', 'Attorney Agent', 'Book Author',
       'Cast Member', 'Commenter', 'Composer', 'Cosponsor', 'Counsel',
       'Interviewer', 'Producer', 'Recipient', 'Reviewed Author',
       'Scriptwriter', 'Words By', 'Guest', 'Number', 'Edition',
       'Running Time', 'Scale', 'Medium', 'Artwork Size', 'Filing Date',
       'Applicatio

In [ ]:
def data_cleaning(df):
    df = df.astype(str)
    extra_feature_list = ['Manual Tags', 'Automatic Tags','Translator', 
        'Call Number', 'Extra', 'Notes', 'File Attachments', 'Link Attachments', 
        'Place', 'Language', 'Rights', 'Type', 'Archive', 'Archive Location', 
        'Library Catalog','Contributor', 'Attorney Agent', 'Book Author',
        'Cast Member', 'Commenter', 'Composer', 'Cosponsor', 'Counsel',
        'Interviewer', 'Producer', 'Recipient', 'Reviewed Author',
        'Scriptwriter', 'Words By', 'Guest', 'Number', 'Edition',
        'Running Time', 'Scale', 'Medium', 'Artwork Size', 'Filing Date',
        'Application Number', 'Assignee', 'Issuing Authority', 'Country',
        'Meeting Name', 'Conference Name', 'Court', 'References', 'Reporter',
        'Legal Status', 'Priority Numbers', 'Programming Language', 'Version',
        'System', 'Code', 'Code Number', 'Section', 'Session', 'Committee',
        'History', 'Legislative Body']
    columns_to_drop = []
    for extra_feature in df.columns:
        if extra_feature in extra_feature_list:
            columns_to_drop.append(extra_feature)
    df.drop(columns=columns_to_drop, inplace=True)
    add_attributes(df)
    display(df)
    nan_df = df[df['ISBN'].isna()]
    isbn_df = df[~df['ISBN'].isna()]

    isbn_df.loc[:, 'ISBN'] = isbn_df['ISBN'].str.replace('-', '', regex=False)
    isbn_df.loc[:, 'ISBN'] = isbn_df['ISBN'].apply(modify_isbn)

    broken_isbn_df = isbn_df[~isbn_df['ISBN'].str[:3].isin(['978', '979'])]
    whole_isbn_df = isbn_df[isbn_df['ISBN'].str[:3].isin(['978', '979'])]

    broken_isbn_df.loc[:,'ISBN'] = broken_isbn_df['ISBN'].apply(lambda x: '978' + str(x)[:10]) # Is 978 the right choice here??

    isbn_df = pd.concat([broken_isbn_df, whole_isbn_df])

    isbn_df = isbn_df.astype(str)

    # # Drop duplicates based on relevant columns while keeping the first occurrence
    isbn_df_unique = isbn_df.drop_duplicates(subset=['ISBN'], keep='first')

    # # Group by 'ISBN' and combine the 'Subject' values
    isbn_df_unique = isbn_df_unique.groupby('ISBN').agg({
        'Series Title': 'first',  # Keep the first value (they are the same for each ISBN)
        'Title/Subtitle': 'first',  # Same as above
        'ISSN': 'first',  # Same as above
        'Author': 'first',  # Same as above
        'Publication Year': 'first',  # Same as above
        'Publisher': 'first',  # Same as above
        'Material Type': 'first',  # Same as above
        'Lexile': 'first',  # Same as above
        'Subject': ', '.join  # Combine all unique 'Subject' values
    }).reset_index()

    for isbn in isbn_df_unique['ISBN']:
        if pd.isna(isbn):
            continue  # Skip if ISBN is NaN
        url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
        response = requests.get(url)
        if response.status_code == 200: # Confirm request worked/is available
            book_data = response.json()
            if 'items' in book_data:
            # Extract book details from the API response
                book_info = book_data['items'][0]['volumeInfo']
                # Missing:
                # Series name and position
                # Form
                # Format
                book_details = {
                    'ISBN#': isbn,
                    'Title': book_info.get('title', np.nan),
                    'Subtitle': np.nan, # Not available
                    'Authors': ', '.join(book_info.get('authors', ['N/A'])),
                    'Publisher': book_info.get('publisher', np.nan),
                    'PublishedDate': book_info.get('publishedDate', np.nan),
                    'CopyrightDate': np.nan, # Not available
                    'Summary': book_info.get('description', 'No description available'), # NEED
                    'Genre': ', '.join(book_info.get('categories', ['N/A'])), # NEED
                    'PageCount': book_info.get('pageCount', np.nan), # NEED
                    'Type': book_info.get('printType', np.nan), # Need
                    'Categories': book_info.get('Categories', np.nan)
                    # Type of book - ficiton, nonfiction, blended
                }
                # Accessing the correct row using ISBN in the DataFrame
                row_index = isbn_df_unique[isbn_df_unique['ISBN'] == isbn].index
                
                if not row_index.empty:  # Check if a matching ISBN was found
                    row_index = row_index[0]  # Get the first (and expected only) match

                    # Only update if the current value is NaN
                    if pd.isna(isbn_df_unique.loc[row_index, 'Title/Subtitle']):
                        isbn_df_unique.loc[row_index, 'Title/Subtitle'] = book_details['Title']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Author']):
                        isbn_df_unique.loc[row_index, 'Author'] = book_details['Authors']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Publication Year']):
                        isbn_df_unique.loc[row_index, 'Publication Year'] = book_details['PublishedDate']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Publisher']):
                        isbn_df_unique.loc[row_index, 'Publisher'] = book_details['Publisher']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Material Type']):
                        isbn_df_unique.loc[row_index, 'Material Type'] = book_details['Type']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Subject']):
                        isbn_df_unique.loc[row_index, 'Subject'] = book_details['Genre']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Summary']):
                        isbn_df_unique.loc[row_index, 'Summary'] = book_details['Summary']
                    if pd.isna(isbn_df_unique.loc[row_index, 'Page Count']):
                        isbn_df_unique.loc[row_index, 'Page Count'] = book_details['PageCount']

    complete_df = pd.concat([isbn_df_unique, nan_df])
    return complete_df

In [124]:
test = data_cleaning(blueHill)

,Key,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,DOI,Url,...,Summary,Series Name/Position,Genre,Form,Format,Page Count,Type,Material Type,Subject,Lexile
0,BBF95TEY,book,nan,Author,Title,nan,nan,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HC4M3PQB,book,2003.0,"Watts, Barrie",Butterfly,nan,1-58340-234-9,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,YQQ39KTF,book,2003.0,"Watts, Barrie",Sunflower,nan,1-58340-232-2,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HDF6WZIW,book,2003.0,"Watts, Barrie",Mouse,nan,1-58340-231-4,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BV6WGJJK,book,2002.0,"Watts, Barrie",Snake,nan,1-58340-200-4,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11288,RW5ZL2CB,book,2022.0,"Knox, Kelly",Be more obi-wan: Navigate your world with wit ...,nan,978-0-7440-7772-8,nan,nan,http://link.overdrive.com/?websiteID=202788&ti...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11289,2KEH6KYZ,book,2021.0,"Horton, Cole",Star wars: battles that changed the galaxy,nan,978-0-7440-5740-9,nan,nan,http://link.overdrive.com/?websiteID=202788&ti...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11290,QEYQL3MM,book,2007.0,"Piven, |Hanokh",My dog is as smelly as dirty socks: and other ...,nan,978-0-375-84052-4,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11291,GD5RV9GW,book,2009.0,"Hahn, Mary Downing",Closed for the season: a mystery story,nan,978-0-547-39853-2,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


KeyboardInterrupt: 

In [61]:
test

,ISBN,Series Title,Title/Subtitle,ISSN,Author,Publication Year,Publisher,Material Type,Lexile,Subject,...,Committee,History,Legislative Body,Copyright Date,Summary/Synopsis,Series Name/Position,Genre,Form,Format,Page Count
0,9780007150350,nan,nan,nan,"Jeffers, Oliver",2005.0,HarperCollins Children's Books,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9780020420307,nan,nan,nan,"Stevenson, Augusta",1986.0,Aladdin,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9780020442009,nan,nan,nan,"Lewis, C. S.",1970.0,Collier,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9780020442106,nan,nan,nan,"Lewis, C. S.",1970.0,Collier,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,978002063420X,nan,nan,nan,"Mohlenbrock, Robert H.",1987.0,Macmillan,nan,nan,nan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10711,NaN,NaN,NaN,NaN,"Burton, Virginia Lee",1962.0,Houghton,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10787,NaN,NaN,NaN,NaN,"Krensky, Stephen",1989.0,Random House,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11116,NaN,NaN,NaN,NaN,"Angleberger, Tom",2013.0,Amulet Books,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11165,NaN,NaN,NaN,NaN,"Warner, Gertrude Chandler",1962.0,A. Whitman,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
blueHill.drop([0], inplace=True)

In [31]:
blueHill.columns

Index(['Key', 'Item Type', 'Publication Year', 'Author', 'Title',
       'Publication Title', 'ISBN', 'ISSN', 'DOI', 'Url', 'Abstract Note',
       'Date', 'Date Added', 'Date Modified', 'Access Date', 'Pages',
       'Num Pages', 'Issue', 'Volume', 'Number Of Volumes',
       'Journal Abbreviation', 'Short Title', 'Series', 'Series Number',
       'Series Text', 'Series Title', 'Publisher', 'Place', 'Language',
       'Rights', 'Type', 'Archive', 'Archive Location', 'Library Catalog',
       'Call Number', 'Extra', 'Notes', 'File Attachments', 'Link Attachments',
       'Manual Tags', 'Automatic Tags', 'Editor', 'Series Editor',
       'Translator', 'Contributor', 'Attorney Agent', 'Book Author',
       'Cast Member', 'Commenter', 'Composer', 'Cosponsor', 'Counsel',
       'Interviewer', 'Producer', 'Recipient', 'Reviewed Author',
       'Scriptwriter', 'Words By', 'Guest', 'Number', 'Edition',
       'Running Time', 'Scale', 'Medium', 'Artwork Size', 'Filing Date',
       'Applicatio

In [ ]:
test = blueHill['Series Title']

<bound method Series.dropna of 1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
         ..
11288   NaN
11289   NaN
11290   NaN
11291   NaN
11292   NaN
Name: Series Title, Length: 11292, dtype: float64>

In [39]:
blueHill = blueHill[['Item Type', 'Publication Year', 'Author', 'Title', 'Publication Title', 'ISBN', 'Abstract Note', 'Pages', 'Num Pages', 'Issue', 
         'Volume', 'Series', 'Series Number', 'Series Title', 'Publisher', 'Editor', 'Series Editor', 'Contributor', 'Book Author']]

In [40]:
add_attributes(blueHill)
blueHill

/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_82821/3289360659.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,category] = np.nan
/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_82821/3289360659.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,category] = np.nan
/var/folders/k6/lr3d617147z0f_fh76hx2dm40000gp/T/ipykernel_82821/3289360659.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,Item Type,Publication Year,Author,Title,Publication Title,ISBN,Abstract Note,Pages,Num Pages,Issue,...,Copyright Date,Summary/Synopsis,Series Name/Position,Genre,Form,Format,Page Count,Type,Material Type,Subject
1,book,2003.0,"Watts, Barrie",Butterfly,NaN,1-58340-234-9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,book,2003.0,"Watts, Barrie",Sunflower,NaN,1-58340-232-2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,book,2003.0,"Watts, Barrie",Mouse,NaN,1-58340-231-4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,book,2002.0,"Watts, Barrie",Snake,NaN,1-58340-200-4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,book,2002.0,"Watts, Barrie",Duck,NaN,1-58340-197-0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11288,book,2022.0,"Knox, Kelly",Be more obi-wan: Navigate your world with wit ...,NaN,978-0-7440-7772-8,A fun ebook with inspiration from the galaxy's...,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11289,book,2021.0,"Horton, Cole",Star wars: battles that changed the galaxy,NaN,978-0-7440-5740-9,"Explore the ""Wars"" in Star Wars as never befor...",NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11290,book,2007.0,"Piven, |Hanokh",My dog is as smelly as dirty socks: and other ...,NaN,978-0-375-84052-4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11291,book,2009.0,"Hahn, Mary Downing",Closed for the season: a mystery story,NaN,978-0-547-39853-2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
